In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import itertools

In [ ]:
!pip install pandas==1.3.0

In [ ]:
!pip install pip install tweet-preprocessor
!pip install pycountry
# !pip install pandas --upgrade
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
!pip install scipy --upgrade
import ipdb
%pdb off
!pip install tensorflow --upgrade


In [ ]:
import pandas as pd
import numpy as np
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import json
import pickle
import joblib
import torch
import preprocessor as p
import pycountry
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import expit

from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

nltk.download('omw-1.4')

In [ ]:
device = torch.device("cuda")

In [ ]:
def load_topic_model(user_feeds):
        global topic_classes

        MODEL = f"cardiffnlp/tweet-topic-21-multi"

        with torch.no_grad():
            topic_model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)

        topic_classes = topic_model.config.id2label#

        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        tokens = user_feeds.clean_text.apply(lambda row: tokenizer(row, return_tensors='pt'))

        print('loaded topic model')
        print(tokens)
        return topic_model , tokens

In [ ]:
def load_psysentimento_model(user_feeds):
        
        tweets = user_feeds.clean_text.to_list()
        analyzer = create_analyzer(task="hate_speech", lang="en")
        hate_labels = ['hateful', 'targeted', 'aggressive']

        hate_out = [ analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]

        e_analyzer = create_analyzer(task="emotion", lang="en")
        emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     

        emo_out = [ e_analyzer.predict(preprocess_tweet(txt)) for txt in tweets ]

        return hate_out, emo_out

In [ ]:
def tweet_cleaner(tw_list):
        remove_rt = lambda x: re.sub('RT @\w+: '," ",str(x))
        rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(x))
        hash = lambda x: re.sub(r'#', "", str(x))
        amp = lambda x: re.sub(r'&amp', "", str(x))

        print("tweet_cleaner")
        print(tw_list[:10])
        tw_list['grammartext'] = tw_list['text'].map(remove_rt).map(rt)
        print("grammar")
        tw_list['clean_text'] = tw_list.text.map(remove_rt).map(rt)
        print("clean_text")
        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
        tw_list["grammartext"] = tw_list.grammartext.map(p.clean)
        print("grammar")

        p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.NUMBER)
        tw_list["clean_text"] = tw_list.clean_text.map(p.clean).map(hash).map(amp)
        tw_list["clean_text"] = tw_list.clean_text.str.lower()
        return tw_list

In [ ]:
def get_sentiment(row):
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']
        if pos< neg:       
            label = "negative"
        elif pos > neg:       
            label = "positive"
        else:      
            label = "neutral"
        return [label, neg,neu,pos, comp]



def get_topic(topic_model, row):
        # tokens = self.topic_tokenizer(row.clean_text,return_tensors='pt')
        output = topic_model(**row.to(device))
        scores = output[0][0].detach().cpu().numpy()
        scores = expit(scores)
        pred = np.argmax(scores)
        return [pred] + scores.flatten().tolist()


def get_hate(row):
        hate_labels = ['hateful', 'targeted', 'aggressive']
        return [row.probas[hate_labels[i]] for i in range(3) ]

def get_emo(row):
        emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear'] 
        return [row.probas[emo_labels[i]] for i in range(7) ]


def get_readability(row):
        if not row:
            return [0]*23
        else:
            results = readability.getmeasures(row,lang='en')
            # [ df.loc[index, score] = results['readability grades'][score] for score in \
            #  ['Kincaid','ARI', 'Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', \
            #   'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex'] ]

            return [ grade[t] for grade in [results['readability grades'], results['sentence info'] ] for t in grade ]


In [ ]:

hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            # 'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
# add back            
# hashtags = ['F1']

for hash in hashtags:
        df = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/raw_data/{hash}.csv') 
        #id,	text,	created_at,	user_id,	followers_count,	friends_count	,favourites_count,	retweet_count	
        print(df[df['user_id'].isnull()])
        df['user_id'] = pd.to_numeric(df['user_id'], errors='coerce')
        df = df.dropna(subset=['user_id'])
        df['user_id'] = df['user_id'].astype(int)
        df['user_id']= df['user_id'].map(str)
        
        user_feeds = tweet_cleaner(df)
        topic_model, topic_in = load_topic_model(user_feeds)
        hate, emo = load_psysentimento_model(user_feeds)

        read_cols = ['Kincaid', 'ARI', 'Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', 
                    'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex','characters_per_word', 'syll_per_word', 
                    'words_per_sentence', 'sentences_per_paragraph', 'type_token_ratio', 'characters', 'syllables', 
                    'words', 'wordtypes', 'sentences', 'paragraphs', 'long_words', 'complex_words', 'complex_words_dc']    



        sent = user_feeds.copy().clean_text.to_list()
        grammer_in = user_feeds.grammartext.tolist()
        read_in = user_feeds.clean_text.tolist()


        sent = user_feeds.copy().clean_text.to_list()
        grammer_in = user_feeds.grammartext.tolist()
        read_in = user_feeds.clean_text.tolist()
        
        sent_output = [ get_sentiment(row) for row in sent]
        print('got sentiment')

        topic_output = [ get_topic(topic_model, row) for row in topic_in]
        print('got topic')

        hate_output = [ get_hate(row) for row in hate]
        print('got hate ')

        hate_output = abs(np.array(hate_output) - np.mean(hate_output, axis=0)).tolist() # fix error in hate classification
        hate_output = abs(np.array(hate_output) - np.mean(hate_output, axis=0)).tolist()

        emo_output = [ get_emo(row) for row in emo]
        print('got emo')

        read_out = [ get_readability(row) for row in read_in]
        print('got readability scores ')


        
        # PLACING ALL INTO THE USER FEEDS DF!!!!!!
        all = np.hstack((sent_output,topic_output,hate_output,emo_output, read_out))

        topics = [ val for _,val in topic_classes.items()]

        hate_labels = ['hateful', 'targeted', 'aggressive']
        emo_labels = ['joy','sadness','others','anger','surprise','disgust','fear']     

        cols = ['sentiment','neg','neu','pos','comp','topic'] + topics + hate_labels + emo_labels + read_cols

        user_feeds_df = pd.DataFrame(all.tolist())
        user_feeds_df.columns = cols


        user_feeds_df['id'] = list(user_feeds.index.values)
        user_feeds_df['user_id'] = list(user_feeds['user_id'].tolist())


        int_cols = ['neg','neu','pos','comp'] + topics + hate_labels + emo_labels + read_cols 

        user_feeds_df[int_cols] = user_feeds_df.copy()[int_cols].astype(float)

        user_feeds = user_feeds_df.copy()

        del user_feeds_df

        fid = list(set(user_feeds['user_id'].tolist()))

        df = df.copy()[df['user_id'].isin(fid)]

        user_ids = df['user_id'].tolist()

        hates = ['hateful', 'targeted', 'aggressive']
        emos = ['joy','sadness','others','anger','surprise','disgust','fear']     

        del topic_classes, hate_labels, emo_labels        

        score_cols = ['user_id','neg','neu','pos','comp']+topics + hates + emos + read_cols

        score_df = user_feeds[score_cols]

        label_df = user_feeds[['user_id','sentiment','topic']]

        def Average(lst):
            return sum(lst) / len(lst)
          
        import random
        from collections import Counter
        from itertools import groupby
        def cust_mode(l):
            freqs = groupby(Counter(l).most_common(), lambda x:x[1])
            return [val for val,count in next(freqs)[1]]

        all_s = []
        all_mode = []
        all_count = []

        for user in user_ids:

            s_df = score_df[score_df['user_id']==user].drop('user_id',axis=1).values.tolist()
            
            l_df = label_df[label_df['user_id']==user].drop('user_id',axis=1).values.tolist()

            all_s.append([ Average(x) for x in zip(*s_df) ])

            all_mode.append( [ random.choice(cust_mode(x)) for x in zip(*l_df)] )

            counts = [ x for x in zip(*l_df) ] 

            sent_count = [ counts[0].count(s) for s in ['negative','neutral','positive'] ]
            topic_count = [ counts[1].count(str(float(s))) for s in range(19) ]

            all_count.append(sent_count + topic_count)


        new_cols = [ f'user_{x}_mean' for x in ['neg','neu','pos','comp']+topics + hates + emos + read_cols ]# + ['politeness'] ]

        df[new_cols]  = all_s

        new_col = [ f'user_{x}_mode' for x in ['sentiment','topic']]

        df[new_col] = all_mode

        df.reset_index(drop=False)
        df.set_index('id', inplace = True)

        df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/1/{hash}_tweet_features.csv')

In [ ]:
df['user_id'][22]